In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

<h1>P-Hacking and Multiple Comparisons Bias</h1>

Multiple comparisons bias is a pervasive problem in statistics, data science, and in general forecasting/predictions. The short explanation is that the more tests you run, the more likely you are to get an outcome that you want/expect. If you ignore the multitude of tests that failed, you are clearly setting yourself up for failure by misinterpreting what's going on in your data.

A particularly common example of this is when looking for relationships in large data sets comprising of many independent series or variables. In this case you run a test each time you evaluate whether a relationship exists between a set of variables.

<b>p-value Refresher</b>

What's important to remember is they're used to test a hypothesis given some data. Here we are testing the hypothesis that a relationship exists between two series given the series values.

<b>IMPORTANT: p-values must be treated as binary.</b>

A common mistake is that p-values are treated as more or less significant. This is bad practice as it allows for what's known as p-hacking and will result in more false positives than you expect. Effectively, you will be too likely to convince yourself that relationships exist in your data.

To treat p-values as binary, a cutoff must be set in advance. Then the p-value must be compared with the cutoff and treated as significant/not signficant. Here we'll show this.

<b>The Cutoff is our Significance Level</b>

We can refer to the cutoff as our significance level because a lower cutoff means that results which pass it are significant at a higher level of confidence. So if you have a cutoff of 0.05, then even on random data 5% of tests will pass based on chance. A cutoff of 0.01 reduces this to 1%, which is a more stringent test. We can therefore have more confidence in our results.

<h1>Experiment</h1>

We'll start by defining a data frame.

Now we'll populate it by adding N randomly generated timeseries of length T.

In [2]:
df = pd.DataFrame()

N = 20
T = 100

for i in range(N):
    X = np.random.normal(0, 1, T)
    X = pd.Series(X)
    name = 'X%s' % i
    df[name] = X

In [3]:
df.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19
0,1.413906,-0.323798,-1.295361,1.341572,-1.256528,-0.649136,0.722056,-0.396289,2.504860,-0.736986,2.938516,0.417214,0.963820,0.089137,-2.134278,0.249169,0.264261,-0.375090,-0.375058,0.450061
1,-0.331968,0.177424,0.870572,-0.024689,1.083116,0.124674,1.437415,-1.648218,1.401452,-1.316832,-0.179760,1.100040,-1.069630,-0.380342,-0.034688,-1.205876,0.480328,0.969791,1.195862,-0.173823
2,0.751059,0.440441,-0.456740,-0.821677,1.388364,0.799007,-0.078635,-1.818262,0.700646,0.009169,-1.095863,-1.390747,-0.596436,0.273973,-0.521854,-1.297937,0.657046,-1.359455,-1.236116,0.045265
3,1.264794,0.074138,0.714124,-1.978385,-0.554169,0.968150,1.096083,-0.676375,-0.649699,1.125176,0.852348,0.575840,-0.902550,-1.447900,0.752670,-0.053308,0.771450,0.197808,2.551308,-0.097992
4,-0.309942,-1.523519,-0.087799,-1.231231,-0.445806,0.039958,-2.220166,1.183873,1.527586,-0.157180,0.212755,1.243959,-0.647194,0.592592,0.127682,0.358640,-0.865317,-1.550470,-0.616804,1.753407


Now we'll run a test on all pairs within our data looking for instances where our p-value is below our defined cutoff of 5%.

In [4]:
cutoff = 0.05

significant_pairs = []

for i in range(N):
    for j in range(i+1, N):
        Xi = df.iloc[:, i]
        Xj = df.iloc[:, j]
        
        results = stats.spearmanr(Xi, Xj)
        
        pvalue = results[1]
        
        if pvalue < cutoff:
            significant_pairs.append((i, j))

Before we check how many significant results we got, let's run out some math to check how many we'd expect. The formula for the number of pairs given N series is

$$\frac{N(N−1)}{2}$$
 
There are no relationships in our data as it's all randomly generated. If our test is properly calibrated we should expect a false positive rate of 5% given our 5% cutoff. Therefore we should expect the following number of pairs that achieved significance based on pure random chance.

In [5]:
(N * (N-1) / 2) * 0.05


9.5

Now let's compare to how many we actually found.

In [6]:
len(significant_pairs)

10

<h1>Experiment - Running Many Tests</h1>

Use Spearman Rank Correlation (https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.spearmanr.html) to evaluate the relationship between the pairs of variables. Run it 100 times and report the average number of significant pairs.

Spearman Rank Correlation is a variation of correlation that takes into account the ranks of the data. This can help with weird distributions or outliers that would confuse other measures. The test also returns a p-value, which is key here. A higher coefficient means a stronger estimated relationship.

<h1>Visualizing What's Going On</h1>
     
Visualize the data as a histogram and draw lines in the cutoffs 0.01 and 0.05 (x axis).

<h1>Increase the number of timeseries (N) and run the experiment again</h1>

<h1>What can you conclude with this experiment?</h1>

<h1>Run the experiment again with Bonferroni Correction </h1>

https://en.wikipedia.org/wiki/Bonferroni_correction